# libraries & global vars

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

# output path for merged career

In [2]:
pathout_merged_career_undivided = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 4 - career/4.1 career_undivided_unparsed_uncoded/"

# import combined leaders

In [3]:
## combined leaders to clean - file in
filein_combined_leaders_toclean = "../../data/combined data/combined data - 1 cleaning/cleaning step 1 - leaders/leaders_toparse.xlsx"
merged_leaders = pd.read_excel(filein_combined_leaders_toclean,dtype="str")

# import old career link data - and streamline old_career_link

In [4]:
## 2015 career - raw - file in
file_2015_career_raw = "../../data/2015 data/2015 data - 0 raw/leadercareerlinkmou.csv"
old_career_link_columns = ["MOUID_2015", "Name", "CareerStringID_2015", "CareerString_2015", "CareerStartYear", "CareerStartMonth", "EventType", "JointAppointment", "Org1", "Org2", "Org3", "Org4", "Org5", "Position", "OrgLevel", "PositionLevel", "OrgPositionString"]
old_career_link = pd.read_csv(file_2015_career_raw,sep=";",names=old_career_link_columns,dtype="str")

# recast CareerStringID_2015 as an integer
old_career_link["CareerStringID_2015"] = pd.to_numeric(old_career_link["CareerStringID_2015"])

# add LeaderID to old_family. select columns to merge with new_family
merged_leaders_key = merged_leaders[["LeaderID","MOUID_2015"]]
old_career_link = old_career_link.merge(merged_leaders_key,on="MOUID_2015",how="inner")

# filter out null rows
old_career_link = old_career_link[old_career_link["MOUID_2015"].notna()]

In [5]:
# create old_career_link
old_career_link = old_career_link[["LeaderID","CareerStringID_2015","CareerString_2015"]]
old_career_link.shape

(5707, 3)

# import new career link data - streamline new_career_link

In [6]:
## 2022 career - create new_career_link
file_2022_career_table = "../../data/2022 data/2022 data - 2 tables/career_everything.xlsx"
new_career_link = pd.read_excel(file_2022_career_table,dtype="str")
new_career_link.columns = [item+"_2022" if item != "LeaderID" else item for item in list(new_career_link.columns)]

In [7]:
new_career_link.shape

(7289, 3)

In [8]:
new_career_link[0:5]

,LeaderID,CareerDateString_2022,CareerString_2022
0,강관주,2011.12,김정일 국가장의위원회 위원
1,강관주,2010.09,조선노동당 중앙위원회 후보위원
2,강관주,2010.04,김중린 국가장의위원회 위원
3,강관주,2009,내각 225부 부장
4,강관주,2008.10,박성철 국가장의위원회 위원


# merge old and new career_link, and export

In [9]:
# rename CareerStrings
old_career_link.rename(columns={"CareerString_2015":"CareerString"},inplace=True)
new_career_link.rename(columns={"CareerString_2022":"CareerString"},inplace=True)

In [10]:
career_link = old_career_link.merge(new_career_link,how="outer",on=["LeaderID","CareerString"])

In [11]:
career_link[career_link.CareerDateString_2022.notnull() & career_link.CareerStringID_2015.notnull()]

,LeaderID,CareerStringID_2015,CareerString,CareerDateString_2022
320,김병률,220.0,6.25당시 김일성 호위무관,연도미상
574,문재철,391.0,조선대외문화연락위원회 부위원장,2009.12
698,최광,464.0,1945이전 빨치산 활동,연도미상
947,리태남,637.0,내각 부총리,2010.06
1581,백학림,1024.0,1945이전 빨치산 활동(빨치산 소년대원),연도미상
2301,전하철,1451.0,최고인민회의 제12기 대의원(보선),2010.06
3072,조명록,1875.0,"1945이전 만주 비행학교, 만경대혁명학원",연도미상
3393,박명철a,1451.0,최고인민회의 제12기 대의원(보선),2010.06
3865,태종수,2306.0,당 총무부 부장,2010.06
4260,장병규,2518.0,최고인민회의 제12기 대의원 겸 법제위원회 위원,2010.04


In [12]:
career_link.shape

(12985, 4)

In [13]:
old_career_link.shape[0] + new_career_link.shape[0]

12996

In [14]:
career_link.to_excel(pathout_merged_career_undivided+"career_link_undivided_unparsed_uncoded.xlsx",index=False,encoding="utf-8")

# merge career_str from old and new

In [15]:
career_str = pd.DataFrame(career_link[["CareerString","CareerDateString_2022"]]).drop_duplicates()
career_str.sort_values("CareerString",inplace=True,ignore_index=True)

In [16]:
career_str.shape

(8045, 2)

In [17]:
# create new CareerStringID
career_str["CareerStringID"] = range(0,career_str.shape[0])

In [18]:
career_str

,CareerString,CareerDateString_2022,CareerStringID
0,故 김정일 국가장의위원회 위원,2011.12,0
1,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,1
2,근로단체부장 해임 * 후임: 리일환,2014,2
3,금수산궁전 개관식 참석시 동행,2013.08,3
4,노동성 상(유임),1999.02,4
5,능라인민유원지 준공식 참석시 동행 * ‘부인 리설주 동지’라고 보도(중앙방송),2012.08,5
6,당 정치국 후보위원,2020.01,6
7,당 중앙위원회 정치국 위원 (*당 중앙위원회 제7기 제4차 전원회의에서 보선),2019.04,7
8,대동강과수종합농장과 대동강과일종합가공 공장 시찰시 동행,2014.12,8
9,동해안 감나무중대(女軍) 방문시 동행,2012.09,9


In [19]:
career_str.to_excel(pathout_merged_career_undivided+"career_str_undivided_unparsed_uncoded.xlsx",index=False,encoding="utf-8")